In [251]:
import os
import json
import pandas as pd
import traceback

In [252]:
from langchain.chat_models import ChatOpenAI

In [253]:
from dotenv import load_dotenv

load_dotenv() 

False

In [254]:
Keys=os.getenv("Key")

In [255]:
llm=ChatOpenAI(openai_api_key=Keys,model_name="gpt-3.5-turbo",temperature=0.5)

In [256]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000021168DAA850>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000021168DAEFA0>, temperature=0.5, openai_api_key='sk-proj-RxitJZaEkgNDlHTUPFXjT3BlbkFJuu67bFRx1ux0HOjwqaUW', openai_proxy='')

In [257]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [258]:
RESPONSE_JSON={
    "1":{
        "mcq":"multiple choice questions",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
    "2":{
        "mcq":"multiple choice questions",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
    "3":{
        "mcq":"multiple choice questions",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
    "4":{
        "mcq":"multiple choice questions",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
}

In [259]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [260]:
quiz_generation_prompt=PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)

In [282]:
print(Keys)

sk-proj-RxitJZaEkgNDlHTUPFXjT3BlbkFJuu67bFRx1ux0HOjwqaUW


In [283]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

In [284]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [285]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [286]:
review_chain=LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

In [265]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [266]:
file_path=r"C:\Users\31ash\Desktop\mcq_generator\data.txt"

In [267]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [268]:
print(TEXT)

ChatGPT stands as a pinnacle of technical achievement in the realm of artificial intelligence, boasting an impressive array of capabilities honed through state-of-the-art machine learning techniques. At its core lies a sophisticated neural network architecture, meticulously trained on vast datasets encompassing diverse linguistic patterns and knowledge domains. Through the marvel of deep learning, ChatGPT exhibits a profound understanding of language semantics, syntax, and context, enabling it to comprehend and generate text with remarkable fidelity.

One of its defining features is its ability to engage in contextually relevant conversations, seamlessly tracking the flow of dialogue and adapting responses accordingly. Leveraging techniques such as attention mechanisms and transformer architectures, ChatGPT excels at capturing long-range dependencies within text, facilitating coherent and contextually grounded interactions.

Furthermore, ChatGPT demonstrates impressive language generat

In [269]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "4": {"mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [270]:
NUMBER=5 
SUBJECT="TRIGONOMETRY"
TONE="hard"

In [271]:
#setting up Token usage tracking in langchain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
        "text":TEXT,
        "number":NUMBER,
        "subject":SUBJECT,
        "tone":TONE,
        "response_json":json.dumps(RESPONSE_JSON)
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:ChatGPT stands as a pinnacle of technical achievement in the realm of artificial intelligence, boasting an impressive array of capabilities honed through state-of-the-art machine learning techniques. At its core lies a sophisticated neural network architecture, meticulously trained on vast datasets encompassing diverse linguistic patterns and knowledge domains. Through the marvel of deep learning, ChatGPT exhibits a profound understanding of language semantics, syntax, and context, enabling it to comprehend and generate text with remarkable fidelity.

One of its defining features is its ability to engage in contextually relevant conversations, seamlessly tracking the flow of dialogue and adapting responses accordingly. Leveraging techniques such as attention mechanisms and transformer architectures, ChatGPT excels at capturing long-range dependencies within text, facilitating coh

In [272]:
print(f"total tokens:{cb.total_tokens}")
print(f"Prompt tokens:{cb.prompt_tokens}")
print(f"Completion tokens:{cb.completion_tokens}")
print(f"total cost:{cb.total_cost}")

total tokens:2288
Prompt tokens:1418
Completion tokens:870
total cost:0.003867


In [273]:
quiz=response.get("quiz")

In [274]:
type(quiz)

str

In [275]:
quiz=json.loads(quiz)

In [276]:
quiz

{'1': {'mcq': 'What trigonometric concept is ChatGPT compared to in the text?',
  'options': {'a': 'Sine function',
   'b': 'Cosine function',
   'c': 'Tangent function',
   'd': 'Inverse trigonometric functions'},
  'correct': 'c'},
 '2': {'mcq': 'How is ChatGPT described in terms of language understanding?',
  'options': {'a': 'It struggles with syntax and semantics',
   'b': 'It exhibits a profound understanding of language semantics, syntax, and context',
   'c': 'It is limited to basic vocabulary',
   'd': 'It only generates random text'},
  'correct': 'b'},
 '3': {'mcq': 'What technique does ChatGPT use for capturing long-range dependencies within text?',
  'options': {'a': 'Fourier transform',
   'b': 'Attention mechanisms',
   'c': 'Linear regression',
   'd': 'Support Vector Machines'},
  'correct': 'b'},
 '4': {'mcq': 'What is one of the tasks mentioned that ChatGPT is proficient in?',
  'options': {'a': 'Image recognition',
   'b': 'Sentiment analysis',
   'c': 'Weather pred

In [277]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [278]:
quiz_table_data


[{'MCQ': 'What trigonometric concept is ChatGPT compared to in the text?',
  'Choices': 'a: Sine function | b: Cosine function | c: Tangent function | d: Inverse trigonometric functions',
  'Correct': 'c'},
 {'MCQ': 'How is ChatGPT described in terms of language understanding?',
  'Choices': 'a: It struggles with syntax and semantics | b: It exhibits a profound understanding of language semantics, syntax, and context | c: It is limited to basic vocabulary | d: It only generates random text',
  'Correct': 'b'},
 {'MCQ': 'What technique does ChatGPT use for capturing long-range dependencies within text?',
  'Choices': 'a: Fourier transform | b: Attention mechanisms | c: Linear regression | d: Support Vector Machines',
  'Correct': 'b'},
 {'MCQ': 'What is one of the tasks mentioned that ChatGPT is proficient in?',
  'Choices': 'a: Image recognition | b: Sentiment analysis | c: Weather prediction | d: Music composition',
  'Correct': 'b'},
 {'MCQ': "How does ChatGPT's responsiveness compar

In [279]:
quiz=pd.DataFrame(quiz_table_data)

In [280]:
quiz.to_csv("machinelearning.csv",index=False)

In [281]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'05_15_2024_23_09_33'